In [2]:
from plotly import graph_objects as go
import ipywidgets as widgets
from collections import deque
from copy import deepcopy
import time

In [3]:
def minimaxRoot(depth, board, player):
    # ova ne e alfa beta, gi razgranuva jazlite do kraj i se izvrshuva samo pri prv jazel- prv povik (pri sekoj povik taa tabla mu e pocetna)
    global moves # gi cuva site mozni potezi
    moves = board.available_moves() # go povikuva metodot avalible moves od Board
    scores = {} # prazen recnik za cuvanje na score vo sekoj poteg
    bestMove = -9999 if player else 9999 # Max igrac =1 , min igrac=0

    for move in moves: # iteracija za sekoj poteg vo listata na mozni potezi
        board.push(move) # pravi eden mozen poteg na originalnata tabla, vo for ciklusot 1 po 1 gi pravi site mozni potezi na tablata
        value = minimax(depth - 1, board, -10000, 10000, not player) # go povikuva minimaks ab so namalena dlabocina, eden jazel pomalku ja razgranuvanje ima i go menuva igracot
        scores[move] = value # Go zacuvuva rezultatot vo recnikot scores
        board.pop()  # go vrakja potegot za da ja vrati tablata vo pocetna sostojba
        # OVIE DELOVI ZAKOMENTIRANI SE ALFA-BETA
        #if value == 1000:
         #   bestMove = value
          #  bestMoveFound = move
           # return bestMoveFound , [],[]
        if player and value >= bestMove: # Ako igracot e 1 = Max i ako imame poteg so podobar scorw od -99999
            bestMove = value # bestMove se postavuva na value
            bestMoveFound = move # najdobriot poteg e onoj za koj ispituvame
        elif not player and value <= bestMove: # istoto samo za min
            bestMove = value
            bestMoveFound = move
    
    return bestMoveFound , moves,scores # vrati najdobar poteg,site mozni potezi, recnik za score za sekoj poteg

def minimax(depth, board, alpha, beta, player):
    if depth == 0: # dokolku nema povekje jazli za istazuvanje 
        return board.score() # vrati ja momentalnata sostojba na tabelata

    moves = board.available_moves() # se povikuva avalible moves od board
   # if not moves:
    #    return 1000

    if (player): # ako igracot e MAX
        bestMove = -9999
        for move in moves: # za sekoj poteg vo mmoznite potezi
            board.push(move) # napravi go potegot koj e na red
            bestMove = max(bestMove, minimax(depth - 1, board, alpha, beta, not player)) # rekurzivno povikaj se i vidi dali best move e pogolem od momentalniot
            board.pop() # vrati ja tablata kako shto bese
            alpha = max(alpha, bestMove) # alfa postavi ja na pogolemata vrednost od alpha i best move
            if beta <= alpha: # ako beta e pomalo/ednakvo na alfa
                return bestMove # vrati najdobar poteg koj min kje go izbere, delot so alfa beta potskratuvanje e ovoj
        return bestMove # ako nema takov samo vrati go najdobriot poteg
    else: # Dkolku igracot ne e max mora da e min
        bestMove = 9999
        for move in moves: # za poteg vo moznite potezi
            board.push(move); # napravi go potegot
            bestMove = min(bestMove, minimax(depth - 1, board, alpha, beta, not player)) # rekurzivno povikaj se i vidi dali taa table te vodi do pomal score
            board.pop() # vrati se na pocetnata tabla
            beta = min(beta, bestMove) # izberi go beta da e najmalo mozno
            if beta <= alpha: # dali beta e pomalo ili ednakvo na alfa
                return bestMove # vrati go toj poteg - potkastruvanje
        return bestMove # dokolku ne toa e polosho z amin i vrati go najdobriot poteg

In [4]:
class Board:
    def __init__(self, table, turn=True):
        self.table = deepcopy(table) # kreiranje na kopija za da se izbegne modifikacija na tablata
        self.queue = [] # lista za cuvanje na potezite
        self.turn = turn  # na koj igrac mu e red prv
        
    def push(self,move):
        self.queue.append(move) # dodadi vo stek mozen poteg
        self.update_table() # kako bi izgledala tablata so toj poteg
        self.turn = not self.turn # na drugiot mu e red
        
    def pop(self):
        move = self.queue.pop() # koj e potegot?
        self.update_table(move) # napravi update na board znaejkji ako se izvrshi move
        self.turn = not self.turn # smeni go redot na igracot
        return move # vrati go potegot
        
    def peek(self):
        return self.queue[-1] # koj e posledniot dodaden poteg vo stekot bez da go izvadish
    
    def update_table(self,move=None):
        if move != None: # dokolku ima ushte mozni potezi, stekot ne e prazen
            old, new = move # move e vsushnost torka od stara pozicija i nova pozicija - na 2D table ((),())
            self.table[old[0]][old[1]] = not self.turn #azuriranje na novata pozicija za drugiot igrac
            self.table[new[0]][new[1]] = None if old[1]==new[1] else self.turn # ako novata i starata pozicija se isti - nema poteg; ako ima validen poteg novata pozicija stanuva
            # vrednost na drugiot igrac
        else:
            old, new = self.peek() # najgorniot poteg ne izvaden od stekot so potezi e old,new
            self.table[old[0]][old[1]] = None # starata pozicija e none
            self.table[new[0]][new[1]] = self.turn # novata pozicija e vrednosta na tekovniot igrac
            
    def legal_moves(self,x,y): # prima koordinati, gleda dali e mozen poteg na niv
        x += -1 if self.turn else 1 # ako e na red MAX imame x-1 toa e gore na tabelata, poinaku 1
        moves = []
        for i in range(-1,2):# se dvizi -1,0,1 toa znaci levo desno gore i gore (kako shto se dvizi piunceto)
            if 0 <= x < M and 0 <= y+i < N: # granici na tablata
                if self.table[x][y+i] == None and i==0: # ako e prazno poleto x,y i i=0
                    moves.append((x,y)) # dodadi go potegot vo listata na potezi
                elif self.table[x][y+i] == (not self.turn) and i!=0: # ako pozicijata x,y+i e okupirano od protivnikot (stranicnoto dvizenje)
                    moves.append((x,y+i)) # taa se dodava vo listata na validni pozicii
        return moves # vrati gi site legalni poziciii
    
   
   
    def available_moves(self): # dostapni potezi
        moves = []
        for x in range(M):
            for y in range(N):
                if self.table[x][y] == self.turn: # self.turn e tekoven igrac
                    # gleda dali pozicija x,y e okupirana od tekovniot igrac
                    for move in self.legal_moves(x,y): # dokolku imame piunce na x,y vidi gi legalnite potezi za toa piunce
                        moves.append(((x,y),move)) # dodadi vo listata moves kako (xold,yold),(xnew,ynew)
        return moves # vrati ja listata
    
    def score(self):
        score = 0 
        for x in range(M):
            for y in range(N):
                if self.table[x][y] != None: # dokolku poleto e okupirano od igrac razgleduvame, inaku ne ni e bitno
                    score += M-x if self.table[x][y] == True else -(x+1) # dokolku poleto e zafateno od tekoven igrac score se zgolemuva za M-x
                    # dokolku poleto e zafateno od tugj igrac score se namaluva za x+1
                    if x == 0 and self.table[x][y] == True: # ako igracot e Max (self.table=ture) i se naogja na pozicija nula - prednost 9999
                        # maks pocnuva na pozicija M-1
                        return 9999
                    elif x == M-1 and self.table[x][y] == False: # ako self.table=False-> Min togas za stignuvanje do stranata na MAX
                        # vrati rezultat -9999
                        return -9999
        score += len([i for row in self.table for i in row if i == True]) - len([i for row in self.table for i in row if i == False]) # presmetuva razlika megju tekoven i protivnicki
        # igrac i vrakja score
        return score 



In [5]:
def posible_moves(x,y):
    moves = []
    if TABLE[x][y] == None:
        return moves
    for i in range(-1,2):
        if 0 <= x-1 < M and 0 <= y+i < N:
            if TABLE[x-1][y+i] == None and i==0:
                moves.append((x-1,y)) # da ne moze Max da odi nagore
            elif TABLE[x-1][y+i] == False and i!=0:
                moves.append((x-1,y+i)) # da ne moze min da se spusta od kaj sho dosol
    return moves
def game_end(board):
    if False in board.table[M-1]: # Min stignal do M-1
        return "Red"
    elif True in board.table[0]: # Max stignal do 0
        return "Blue"
    if not board.available_moves():
        return "Red" if board.turn == True else "Blue"
    return None # ushte nikoj ne pobedil

In [6]:
debug = True
moves = None
clicked,xf,yf= 0,0,0
def update_fig(trace, points, selector):
    x,y = points.ys[0],points.xs[0]
    global clicked, xf, yf, moves, TABLE
    clicked +=1  
    
    if clicked == 1:
        xf,yf = x,y
        temp = deepcopy(TABLE)
        moves = posible_moves(x,y)
        if not moves:  clicked = 0
            
        for i,j in moves:
            temp[i][j] = 3 if temp[i][j] == False else 2
            
        colors=['#121212' if n == None 
                else'#ff7f0e' if n == 2 else '#1f77b4' if n == 1 
                else '#d62728' if n == 0 else '#2ca02c' for row in temp for n in row]
        fig.data[0].marker['color'] = colors
        
    elif clicked == 2:
        if (x,y) in moves:
            TABLE[x][y] = True
            TABLE[xf][yf] = None
            
        colors = ['#121212' if n == None else '#1f77b4' if n else '#d62728' for row in TABLE for n in row]
        fig.data[0].marker['color'] = colors
        clicked,moves = 0,None
        
        if game_end(Board(TABLE,False)) != None:
            text.value = f'<font size=4> {game_end(Board(TABLE,False))} wins !!'
            fig.data[0].on_click(None)
            return
        
        time.sleep(0.5) #AI Part
        board = Board(TABLE,False)
        move,moves,scores = minimaxRoot(4, board, False)
        if debug : 
            print([[move,scores[move]] for move in moves])
            
        board.push(move)
        TABLE = board.table
        colors = ['#121212' if n == None else '#1f77b4' if n else '#d62728' for row in TABLE for n in row]
        fig.data[0].marker['color'] = colors
        
        if game_end(Board(TABLE)) != None:
            text.value = f'<font size=4> {game_end(Board(TABLE))} wins !!'
            fig.data[0].on_click(None)
            return
    
def create_fig():
    fig = go.FigureWidget()
    x = [x for y in range(M) for x in range(N)]
    y = [y for y in range(M) for x in range(N)]
    colors = ['#121212' if n == None else '#1f77b4' if n else '#d62728' for row in TABLE for n in row]
    fig.add_scatter(x=x, y=y, mode='markers', marker_size=(M+1)*10+N+2, 
        marker_symbol='square', marker_color=colors,hoverinfo='none')
    fig.data[0].on_click(update_fig)
    fig.update_xaxes(range=[-0.5, N - 0.5], dtick=1, side='top', visible=False,autorange=False)
    fig.update_yaxes(range=[-0.5, M - 0.5], dtick=1, autorange='reversed', visible=False)
    fig.update_layout(width=80*N, height=80*N, showlegend=False,margin={'r':0,'l':0,'t':0,'b':0},plot_bgcolor='#212121')

    return fig

N = 4
M = 4
TABLE = [
    [False,]*N,
    [None,None,None,None],
    [None,None,None,None],
    [True,]*N,
]
fig = create_fig()
text = widgets.HTML(value='<font size=4> Hexapawn')
widgets.VBox([fig,text])

    'data': [{'hoverinfo': 'none',
              'marker': {'color': [#d62728, #…

In [7]:
debug = True
def update_fig(trace, points, selector):
    global TABLE
    
    board = Board(TABLE)
    move,moves,scores = minimaxRoot(4, board, True)
    #move,moves,scores,node_count = minimaxRoot(4, board, True)
    if debug : 
        print([[move,scores[move]] for move in moves])
        #print(f"Minimaks razgrani : {node_count}")

    board.push(move)
    TABLE = board.table
    colors = ['#121212' if n == None else '#1f77b4' if n else '#d62728' for row in TABLE for n in row]
    fig.data[0].marker['color'] = colors

    if game_end(Board(TABLE)) != None:
        text.value = f'<font size=4> {game_end(Board(TABLE))} wins !!'
        fig.data[0].on_click(None)
        return 
    # Tuka se povikuva AI, kako min igrac
    time.sleep(1) #AI Part
    board = Board(TABLE,False)
    move,moves,scores = minimaxRoot(5, board, False)
    if debug : 
        print([[move,scores[move]] for move in moves])

    board.push(move)
    TABLE = board.table
    colors = ['#121212' if n == None else '#1f77b4' if n else '#d62728' for row in TABLE for n in row]
    fig.data[0].marker['color'] = colors

    if game_end(Board(TABLE)) != None:
        text.value = f'<font size=4> {game_end(Board(TABLE))} wins !!'
        fig.data[0].on_click(None)
        return
    
def create_fig():
    fig = go.FigureWidget()
    x = [x for y in range(M) for x in range(N)]
    y = [y for y in range(M) for x in range(N)]
    colors = ['#121212' if n == None else '#1f77b4' if n else '#d62728' for row in TABLE for n in row]
    fig.add_scatter(x=x, y=y, mode='markers', marker_size=(M+1)*10+N+2, 
        marker_symbol='square', marker_color=colors,hoverinfo='none')
    fig.data[0].on_click(update_fig)
    fig.update_xaxes(range=[-0.5, N - 0.5], dtick=1, side='top', visible=False,autorange=False)
    fig.update_yaxes(range=[-0.5, M - 0.5], dtick=1, autorange='reversed', visible=False)
    fig.update_layout(width=80*N, height=80*N, showlegend=False,margin={'r':0,'l':0,'t':0,'b':0},plot_bgcolor='#212121')

    return fig

N = 6
M = 5
TABLE = [
    [False,]*N,
    [None,]*N,
    [None,]*N,
    [None,]*N,
    [True,]*N,
]
fig = create_fig()
text = widgets.HTML(value='<font size=4> Hexapawn')
widgets.VBox([fig,text])

    'data': [{'hoverinfo': 'none',
              'marker': {'color': [#d62728, #…

# BROENJE NA JAZLI

In [18]:
def minimaxRoot(depth, board, player,node_count=0):
    # ova ne e alfa beta, gi razgranuva jazlite do kraj i se izvrshuva samo pri prv jazel- prv povik (pri sekoj povik taa tabla mu e pocetna)
    global moves # gi cuva site mozni potezi
    moves = board.available_moves() # go povikuva metodot avalible moves od Board
    scores = {} # prazen recnik za cuvanje na score vo sekoj poteg
    bestMove = -9999 if player else 9999 # Max igrac =1 , min igrac=0

    for move in moves: # iteracija za sekoj poteg vo listata na mozni potezi
        board.push(move) # pravi eden mozen poteg na originalnata tabla, vo for ciklusot 1 po 1 gi pravi site mozni potezi na tablata
        value,node_count,m,s= minimax(depth - 1, board, -10000, 10000, not player,node_count+1)# go povikuva minimaks ab so namalena dlabocina, eden jazel pomalku ja razgranuvanje ima i go menuva igracot
        print(f"Rekurzivno i vkupno, zosto ne pamtam shto izbrojav prviot pat koga me viknaa nikoj ne mi zacuva vo memorija:(, imam razgraneto :{node_count}")
        print(f"Jas sum od minimaksRoot")
        scores[move] = value # Go zacuvuva rezultatot vo recnikot scores
        board.pop()  # go vrakja potegot za da ja vrati tablata vo pocetna sostojba
        # OVIE DELOVI ZAKOMENTIRANI SE ALFA-BETA
        #if value == 1000:
         #   bestMove = value
          #  bestMoveFound = move
           # return bestMoveFound , [],[]
        if player and value >= bestMove: # Ako igracot e 1 = Max i ako imame poteg so podobar scorw od -99999
            bestMove = value # bestMove se postavuva na value
            bestMoveFound = move # najdobriot poteg e onoj za koj ispituvame
        elif not player and value <= bestMove: # istoto samo za min
            bestMove = value
            bestMoveFound = move
    
    return bestMoveFound ,node_count, moves,scores # vrati najdobar poteg,site mozni potezi, recnik za score za sekoj poteg

def minimax(depth, board, alpha, beta, player):
    if depth == 0: # dokolku nema povekje jazli za istazuvanje 
        return board.score() # vrati ja momentalnata sostojba na tabelata

    moves = board.available_moves() # se povikuva avalible moves od board
   # if not moves:
    #    return 1000

    if (player): # ako igracot e MAX
        bestMove = -9999
        for move in moves: # za sekoj poteg vo mmoznite potezi
            board.push(move) # napravi go potegot koj e na red
            move,node_count=minimax(depth - 1, board, alpha, beta, not player,node_count+1)
            bestMove=max(bestMove,move)
            print(f"Jas razgraniv rekurzivno : {node_count} jazli ")
            print(f"Jas sum od minimaks so alfa beta potsktatuvanje ama mene root me vika")
           # bestMove = max(bestMove, minimax(depth - 1, board, alpha, beta, not player)) # rekurzivno povikaj se i vidi dali best move e pogolem od momentalniot
            board.pop() # vrati ja tablata kako shto bese
            alpha = max(alpha, bestMove) # alfa postavi ja na pogolemata vrednost od alpha i best move
            if beta <= alpha: # ako beta e pomalo/ednakvo na alfa
                return bestMove # vrati najdobar poteg koj min kje go izbere, delot so alfa beta potskratuvanje e ovoj
        return bestMove # ako nema takov samo vrati go najdobriot poteg
    else: # Dkolku igracot ne e max mora da e min
        bestMove = 9999
        for move in moves: # za poteg vo moznite potezi
            board.push(move); # napravi go potegot
            bestMove = min(bestMove, minimax(depth - 1, board, alpha, beta, not player)) # rekurzivno povikaj se i vidi dali taa table te vodi do pomal score
            board.pop() # vrati se na pocetnata tabla
            beta = min(beta, bestMove) # izberi go beta da e najmalo mozno
            if beta <= alpha: # dali beta e pomalo ili ednakvo na alfa
                return bestMove # vrati go toj poteg - potkastruvanje
        return bestMove,node_count # dokolku ne toa e polosho z amin i vrati go najdobriot poteg

Забелешка за броењето јазлиЧ
- Минимакс е рекурзивен алгоритам, значи тој доколку не работи со алфа бета потскратување, доколку се наогја токму на коренот на дрвото че ги разгрании јазлите до долу.
Потоа мин е на ред и го прави истото. Разгранетите состојби не се чуваат никаде во меморијата, па затоа на компјутерот му е неопходно да ги разгранува сите, односно целото дрво.
Доколку има алфа бета потскратување ќе се разгранат јазлите кои не можат да се скратат. Постои и пристам со кој се огрничува длабочината на пребарување, доносно колку длабоко
по дрвото би истражувал минимакс (тоа најчесто се прави кај огромни дрва). Алогирмот е направен така да се брои секој јазол што се разгранува во секој рекурзивен повик, бидејќи 
компјутерот нема информација за разгранувањето што се случувало пред него.


# Dokolku od 0 gi broime site razgraneti jazli , bez razlika na kompjuterska memorija

In [ ]:
def minimaxRoot(depth, board, player):
    global moves, node_count
    moves = board.available_moves()
    scores = {}
    bestMove = -9999 if player else 9999

    for move in moves:
        board.push(move)
        value, node_count = minimax(depth - 1, board, -10000, 10000, not player, node_count)
        scores[move] = value
        board.pop()
        if player and value >= bestMove:
            bestMove = value
            bestMoveFound = move
        elif not player and value <= bestMove:
            bestMove = value
            bestMoveFound = move
    
    return bestMoveFound, moves, scores, node_count

def minimax(depth, board, alpha, beta, player, node_count):
    node_count += 1  # Increment the node count

    if depth == 0:
        return board.score(), node_count

    moves = board.available_moves()

    if player:
        bestMove = -9999
        for move in moves:
            board.push(move)
            value, node_count = minimax(depth - 1, board, alpha, beta, not player, node_count)
            bestMove = max(bestMove, value)
            board.pop()
            alpha = max(alpha, bestMove)
            if beta <= alpha:
                break
        return bestMove, node_count
    else:
        bestMove = 9999
        for move in moves:
            board.push(move)
            value, node_count = minimax(depth - 1, board, alpha, beta, not player, node_count)
            bestMove = min(bestMove, value)
            board.pop()
            beta = min(beta, bestMove)
            if beta <= alpha:
                break
        return bestMove, node_count


# Евалуате стате


In [ ]:
Се игра на табла 4 на 6 4*6=24 полиња со 2 играчи кои имаат 6 пиуна.МИН почнува од 0 , а МАКС почнува од М-1
Значи таблата може да ја поделиме на 4 брана за секој играч
ЗА МИН: 1 ќе носи 0.25 поени, 2 ќе носи 0.5 поени , 3 ќе носи 0.75 поени и 4 носи 1 поен 
ЗА МАКС: 3 носи 0.25, 2 носи 0.5, 1 носи 0.75 и 0 носи 1 поен

Доколку има противнички играч право (2 полиња пред,спроти) нашето пиунче, вредноста на потегот се намалува за 0.7 %
Доколку до новиот потег има дијагонално 2 противнички пиончиња потегот се намалува за 0.9 %
Доколку има дијагонално 1 пионче потегот се намалува за 0.8 %


Секое пионче носи 0.5 поени

Секое земено пионче носи 0.8 поени:
    - Доколку нема друго пионче што може да ја заштити линијата +0.7 поени
    - Доколки не е страшно а нашето пионче ќе се блокира - 0.3 поени
    - Доколку му отвара празна колона на нашето пионче +0.7 поени

In [13]:
def evaluate(state, player, board_size=(4, 6)):
    rows, cols = board_size
    score = 0

    for i, piece in enumerate(state):
        x, y = piece  # координати на пиончето

        # Проверка дали пиончето е на нашиот играч или на противникот
        is_player_piece = i < len(state) // 2 if player == 'MIN' else i >= len(state) // 2

        if is_player_piece:
            # Основна вредност на пиончето
            score += 0.5

            # Позициони бонус според редот (MIN и MAX имаат различни бонуси)
            if player == 'MIN':
                if x == 0:  # Прв ред
                    score += 0.25
                elif x == 1:
                    score += 0.5
                elif x == 2:
                    score += 0.75
                elif x == 3:  # Последен ред
                    score += 1
            elif player == 'MAX':
                if x == 3:  # Прв ред за MAX
                    score += 0.25
                elif x == 2:
                    score += 0.5
                elif x == 1:
                    score += 0.75
                elif x == 0:  # Последен ред за MAX
                    score += 1

            # Проверка за противнички пион пред нас (2 полиња напред)
            if is_enemy_in_front(x, y, state, player, board_size):
                score -= 0.7 / 100

            # Проверка за дијагонално спротивнички пиончиња
            diagonal_penalty = check_diagonal_opponents(x, y, state, player, board_size)
            score -= diagonal_penalty

        else:  # Противнички пиончиња
            score -= 0.5

    return score


# Помошни функции
def is_enemy_in_front(x, y, state, player, board_size):
    rows, cols = board_size
    for i, (ex, ey) in enumerate(state):
        is_enemy = i >= len(state) // 2 if player == 'MIN' else i < len(state) // 2
        if is_enemy and abs(ex - x) == 2 and ey == y:  # 2 полиња напред
            return True
    return False


def check_diagonal_opponents(x, y, state, player, board_size):
    rows, cols = board_size
    diagonal_penalty = 0
    for i, (ex, ey) in enumerate(state):
        is_enemy = i >= len(state) // 2 if player == 'MIN' else i < len(state) // 2
        if is_enemy:
            if abs(ex - x) == 1 and abs(ey - y) == 1:  # Дијагонално 1
                diagonal_penalty += 0.8 / 100
            elif abs(ex - x) == 2 and abs(ey - y) == 2:  # Дијагонално 2
                diagonal_penalty += 0.9 / 100
    return diagonal_penalty


In [14]:
# Примерно state:
state = ((0, 0), (0, 1), (1, 2), (3, 5), (2, 3), (3, 4))  # Координати на пиончињата
player = 'MIN'
board_size = (4,3)

score = evaluate(state, player, board_size)
print("Оценка за состојбата:", score)


Оценка за состојбата: 0.9740000000000002


# MINIMAKS IZVADEN ZA DA PECATI

In [35]:
def game_rules(state, move, player):
    """
    Оваа функција ги дефинира правилата на играта:
    - Движење на пиуните за секој играч.
    - Земање противнички пион ако е дијагонално достапен.
    - Проверка дали потегот е валиден.
    
    Влез:
    - `state`: моментална состојба на таблата (торка од торки).
    - `move`: целна позиција за потегот (ред, колона).
    - `player`: тековниот играч (1 за MAX, 0 за MIN).

    Враќа:
    - Новата состојба на таблата (торка од торки).
    """
    state = [list(row) for row in state]
    row, col = move
    direction = -1 if player == 1 else 1  # -1 за MAX (нагоре), 1 за MIN (надолу)
    
    # Проверка дали полето е валидно
    if not (0 <= row < len(state) and 0 <= col < len(state[0])):
        raise ValueError("Невалиден потег: Надвор од границите на таблата.")
    if state[row][col] != -1:
        raise ValueError("Невалиден потег: Полето не е празно.")
    
    # Проверка за валидност на потегот (директно напред или дијагонално ако има противник)
    valid_move = False
    if 0 <= row - direction < len(state):  # Напредно движење
        if state[row - direction][col] == -1:
            valid_move = True
    if 0 <= row - direction < len(state) and 0 <= col - 1 < len(state[0]):  # Лева дијагонала
        if state[row - direction][col - 1] == 1 - player:
            valid_move = True
    if 0 <= row - direction < len(state) and 0 <= col + 1 < len(state[0]):  # Десна дијагонала
        if state[row - direction][col + 1] == 1 - player:
            valid_move = True

    if not valid_move:
        raise ValueError("Невалиден потег: Потегот не е дозволен според правилата.")
    
    # Ажурирање на таблата
    state[row][col] = player
    return tuple(tuple(row) for row in state)
    def available_moves(state, player):
    """
    Генерира валидни потези за тековниот играч со проверка на правилата.
    """
    moves = []
    direction = -1 if player == 1 else 1  # -1 за MAX (нагоре), 1 за MIN (надолу)
    for row in range(len(state)):
        for col in range(len(state[0])):
            if state[row][col] == player:
                # Проверка за напредно движење
                if 0 <= row + direction < len(state) and state[row + direction][col] == -1:
                    moves.append((row + direction, col))
                # Проверка за лева дијагонала
                if 0 <= row + direction < len(state) and 0 <= col - 1 < len(state[0]) and state[row + direction][col - 1] == 1 - player:
                    moves.append((row + direction, col - 1))
                # Проверка за десна дијагонала
                if 0 <= row + direction < len(state) and 0 <= col + 1 < len(state[0]) and state[row + direction][col + 1] == 1 - player:
                    moves.append((row + direction, col + 1))
    
    # Филтрирање потези преку `game_rules` за дополнителна проверка
    valid_moves = []
    for move in moves:
        try:
            game_rules(state, move, player)  # Пробај го потегот
            valid_moves.append(move)        # Ако е успешен, додај го
        except ValueError:
            continue  # Игнорирај невалидни потези
    return valid_moves


def minimax(state, depth, player, alpha, beta, node_count=0):
    """
    Минимакс алгоритам со алфа-бета отсекување, без `evaluate_state`.
    - `state`: моментална состојба како торка од торки.
    - `depth`: преостаната длабочина за истражување.
    - `player`: играч кој прави потегот (1 за MAX, 0 за MIN).
    - `alpha`: алфа вредност за отсекување.
    - `beta`: бета вредност за отсекување.
    - `node_count`: бројач на разгранетите јазли.
    
    Враќа: вредноста на состојбата и бројот на разгранети јазли.
    """
    node_count += 1
    print(f"Јазел {node_count}: Играч {player}, Длабочина {depth}")

    # Базен случај: достигната е длабочина 0
    if depth == 0:
        return 0, node_count  # Оценката е 0 за илустративни цели

    moves = [(row, col) for row in range(len(state)) for col in range(len(state[0])) if state[row][col] == -1]

    if player == 1:  # MAX играчот
        best_value = float('-inf')
        for move in moves:
            new_state = game_rules(state, move, player)
            value, node_count = minimax(new_state, depth - 1, 1 - player, alpha, beta, node_count)
            best_value = max(best_value, value)
            alpha = max(alpha, best_value)
            if beta <= alpha:  # Отсекување
                print(f"Алфа-бета отсекување за MAX при потег {move}, јазел {node_count}")
                break
        return best_value, node_count
    else:  # MIN играчот
        best_value = float('inf')
        for move in moves:
            new_state = game_rules(state, move, player)
            value, node_count = minimax(new_state, depth - 1, 1 - player, alpha, beta, node_count)
            best_value = min(best_value, value)
            beta = min(beta, best_value)
            if beta <= alpha:  # Отсекување
                print(f"Алфа-бета отсекување за MIN при потег {move}, јазел {node_count}")
                break
        return best_value, node_count


# Пример за тестирање
state = (
    (-1, -1, -1, -1, -1, -1),
    (-1, -1, -1, -1, -1, -1),
    (-1, -1, -1, -1, -1, -1),
    (-1, -1, -1, -1, -1, -1)
)

depth = 3
player = 1  # MAX играч

best_value, node_count = minimax(state, depth, player, float('-inf'), float('inf'))
print(f"Најдобра вредност: {best_value}")
print(f"Вкупно разгранети јазли: {node_count}")
initial_state = (
    (1, 1, 1, 1, 1, 1),  # MAX играчот (1)
    (1, 1, 1, 1, 1, 1),
    (-1, -1, -1, -1, -1, -1),
    (-1, -1, -1, -1, -1, -1),
    (0, 0, 0, 0, 0, 0),  # MIN играчот (0)
    (0, 0, 0, 0, 0, 0),
)

# Пример потег
try:
    updated_state = game_rules(initial_state, (2, 3), 1)  # MAX се движи на (2, 3)
    for row in updated_state:
        print(row)
except ValueError as e:
    print(e)

IndentationError: expected an indented block after function definition on line 44 (2630345239.py, line 45)

In [ ]:
def game_rules(state, move, player):
    """
    Оваа функција ги дефинира правилата на играта:
    - Движење на пиуните за секој играч.
    - Земање противнички пион ако е дијагонално достапен.
    - Проверка дали потегот е валиден.
    """
    state = [list(row) for row in state]
    row, col = move
    direction = -1 if player == 1 else 1  # -1 за MAX (нагоре), 1 за MIN (надолу)
    
    # Проверка дали полето е валидно
    if not (0 <= row < len(state) and 0 <= col < len(state[0])):
        raise ValueError("Невалиден потег: Надвор од границите на таблата.")
    if state[row][col] != -1:
        raise ValueError("Невалиден потег: Полето не е празно.")
    
    # Проверка за валидност на потегот (директно напред или дијагонално ако има противник)
    valid_move = False
    if 0 <= row - direction < len(state):  # Напредно движење
        if state[row - direction][col] == -1:
            valid_move = True
    if 0 <= row - direction < len(state) and 0 <= col - 1 < len(state[0]):  # Лева дијагонала
        if state[row - direction][col - 1] == 1 - player:
            valid_move = True
    if 0 <= row - direction < len(state) and 0 <= col + 1 < len(state[0]):  # Десна дијагонала
        if state[row - direction][col + 1] == 1 - player:
            valid_move = True

    if not valid_move:
        raise ValueError("Невалиден потег: Потегот не е дозволен според правилата.")
    
    # Ажурирање на таблата
    state[row][col] = player
    return tuple(tuple(row) for row in state)
